In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 200

In [3]:
basic = pd.read_csv('./basic.csv', sep = ';')

In [4]:
basic.columns

Index(['date', 'time', 'price', 'shares', 'tax', 'fee', 'type', 'broker',
       'isin'],
      dtype='object')

In [7]:
imp = pd.read_csv('./transactions-from-08022019-to-05042023.csv',sep = ';')

In [8]:
imp

,Datum,Auftrag #,Transaktionen,Symbol,Name,ISIN,Anzahl,Stückpreis,Kosten,Aufgelaufene Zinsen,Nettobetrag,Saldo,Währung
0,31-03-2023 15:00:19,0,Depotgebühren,NaN,NaN,NaN,1.0,20.00,1.54,0.0,-21.54,38.56,CHF
1,28-03-2023 21:32:23,132798818,Kauf,MMM,3M ORD,US88579Y1010,1.0,101.70,0.15,0.0,-101.85,8729.44,USD
2,28-03-2023 21:32:23,132798818,Kauf,MMM,3M ORD,US88579Y1010,3.0,101.70,5.46,0.0,-310.56,8831.29,USD
3,28-03-2023 21:13:08,132798867,Kauf,MMM,3M ORD,US88579Y1010,1.0,101.80,0.15,0.0,-101.95,9141.85,USD
4,28-03-2023 21:13:07,132798867,Kauf,MMM,3M ORD,US88579Y1010,3.0,101.80,5.46,0.0,-310.86,9243.80,USD
5,16-03-2023 14:12:52,0,Dividende,AMAT,APPLIED MATERIAL ORD,US0382221051,1.0,2.60,0.78,0.0,1.82,9554.66,USD
6,14-03-2023 10:31:48,0,Dividende,MMM,3M ORD,US88579Y1010,1.0,15.00,4.50,0.0,10.50,9552.84,USD
7,14-03-2023 09:33:25,0,Dividende,NOVN,NOVARTIS N,CH0012005267,1.0,64.00,22.40,0.0,41.60,60.10,CHF
8,20-02-2023 18:31:51,0,Dividende,AAPL,APPLE ORD,US0378331005,1.0,2.76,0.82,0.0,1.94,9542.34,USD
9,15-02-2023 14:19:33,0,Dividende,TKA,THYSSENKRUPP ORD,DE0007500001,1.0,30.00,7.91,0.0,22.09,4399.02,EUR


In [9]:
mapping_col = {'Datum':'datetime',
               'Auftrag #':'Auftrag #',
               'Transaktionen':'type',
               'Symbol':'Symbol',
               'Name':'Name',
               'ISIN':'isin',
               'Anzahl':'shares',
               'Stückpreis':'price', 
               'Kosten':'fee', 
               'Aufgelaufene Zinsen':'Aufgelaufene Zinsen', 
               'Nettobetrag':'Nettobetrag',
               'Saldo':'Saldo',
               'Währung':'originalcurrency'}

mapping_transactions = {'Dividende':'Dividend',
                        'Kauf':'Buy', 
                        'Rückzahlung':'Rückzahlung', 
                        'Depotgebühren':'Depotgebühren',
                        'Fx-Gutschrift Comp.':'TransferIn', 
                        'Fx-Belastung Comp.':'TransferOut', 
                        'Spin off':'Spin off',
                        'Capital Gain':'Capital Gain', 
                        'Verkauf':'Sell', 
                        'Forex-Gutschrift':'Forex-Gutschrift', 
                        'Forex-Belastung':'Forex-Belastung',
                        'Zins':'Zins',
                        'Interne Titelumbuchung':'Interne Titelumbuchung',
                        'Vergütung':'TransferIn',
                        'Berichtigung Börsengeb.':'Berichtigung Börsengeb.'}

mapping_cash_transactions = {'Dividende':'TransferIn',
                        'Kauf':'TransferOut', 
                        'Rückzahlung':'TransferIn', 
                        'Depotgebühren':'TransferOut',
                        'Fx-Gutschrift Comp.':'TransferIn', 
                        'Fx-Belastung Comp.':'TransferOut', 
                        'Spin off':'Spin off',
                        'Capital Gain':'TransferIn', 
                        'Verkauf':'TransferIn', 
                        'Forex-Gutschrift':'TransferIn', 
                        'Forex-Belastung':'TransferOut',
                        'Zins':'TransferIn',
                        'Interne Titelumbuchung':'Interne Titelumbuchung',
                        'Vergütung':'TransferIn',
                        'Berichtigung Börsengeb.':'Berichtigung Börsengeb.'}

keep_columns = ['datetime',
               'type',
               'isin',
               'shares',
               'price',
               'fee',
               'originalcurrency']

In [10]:
exp = imp.rename(columns=mapping_col)[keep_columns]
exp.replace(to_replace = mapping_cash_transactions.keys(), value = mapping_cash_transactions.values(), inplace = True)
exp['holding'] = ''

exp.query("type in ['TransferIn','TransferOut'] and originalcurrency == 'CHF'").holding
# exp.loc[((exp.type == 'TransferIn')|(exp.type == 'TransferOut')) & (exp.originalcurrency == 'CHF'),'holding'] = 'hld_62fe9bf476362ac5e8b88ed5'
# exp.loc[((exp.type == 'TransferIn')|(exp.type == 'TransferOut')) & (exp.originalcurrency == 'USD'),'holding'] = 'hld_62fe9c0476362ac5e8b88ed8' # USD
# exp.loc[((exp.type == 'TransferIn')|(exp.type == 'TransferOut')) & (exp.originalcurrency == 'EUR'),'holding'] = 'hld_62fe9c1398683c34ff685ac2' # EUR

exp.loc[ (exp.originalcurrency == 'CHF'),'holding'] = 'hld_62fe9bf476362ac5e8b88ed5'
exp.loc[ (exp.originalcurrency == 'USD'),'holding'] = 'hld_62fe9c0476362ac5e8b88ed8' # USD
exp.loc[ (exp.originalcurrency == 'EUR'),'holding'] = 'hld_62fe9c1398683c34ff685ac2' # EUR

exp['datetime'] = pd.to_datetime(exp['datetime'], format='%d-%m-%Y %H:%M:%S')
exp['datetime'] = exp["datetime"].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
exp['currency'] = exp['originalcurrency']
exp['tax'] = 0.0
port = exp[exp['type'].isin(['Buy','Sell','Dividend','TransferIn','TransferOut'])]
cash = exp
cash.replace(to_replace = mapping_transactions.keys(), value = mapping_transactions.values(), inplace = True)
cash = cash[['datetime','type','shares','price','fee','holding','tax']]
cash = cash[cash['type'].isin(['TransferIn','TransferOut'])]

cash.to_csv('./cash.csv',sep=';', index = False)

exp.to_csv('./portfolio.csv',sep=';')